In [ ]:
import numpy as np
import snorkel
from snorkel.labeling import labeling_function
from snorkel.labeling import LabelingFunction
from snorkel.labeling import PandasLFApplier
from snorkel.labeling import LFAnalysis
import re
from snorkel.labeling.model import MajorityLabelVoter
import json
import pandas as pd
import label_improve
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

: 

In [ ]:
%load_ext autoreload
%autoreload 2
import label_improve as li

: 

In [ ]:
# Loading the data 
dataset_name = "chemprot"
idx_to_label = json.load(open(f"../weak_datasets/{dataset_name}/label.json"))
label_to_idx = {l:i for i,l in idx_to_label.items()}
valid_df = li.chemprot_to_df(json.load(open(f"../weak_datasets/{dataset_name}/valid.json", "r")))
train_df = li.chemprot_to_df(json.load(open(f"../weak_datasets/{dataset_name}/train.json", "r")))
test_df = li.chemprot_to_df(json.load(open(f"../weak_datasets/{dataset_name}/test.json", "r")))

# Sample a dev set to help seed ideas for LFs
dev_df = train_df.sample(250, random_state=123)

: 

In [ ]:

# chemprot functions:

ABSTAIN = -1
### Keyword based labeling functions ###

## Part of
#0
@labeling_function()
def lf_amino_acid(x):
    return 0 if 'amino acid' in x.text.lower() else ABSTAIN
#1
@labeling_function()
def lf_replace(x):
    return 0 if 'replace' in x.text.lower() else ABSTAIN
#2 TODO: 0.1988
@labeling_function()
def lf_mutant(x):
    def find_word_index(words, target):
        for i, word in enumerate(words):
            if target in word:
                return i
        return -1

    words = x.text.lower().split()
    if any('mutant' in word or 'mutat' in word for word in words):
        # if mutant is between the two entities
        if x.entity1_index == -1 or x.entity2_index == -1:
            return ABSTAIN
        if isinstance(x.entity1_index, int) and isinstance(x.entity2_index, int):
            if x.entity1_index < x.entity2_index:
                if any('mutant' in word or 'mutat' in word for word in words[x.entity1_index:x.entity2_index]):
                    return 0
            else:
                if any('mutant' in word or 'mutat' in word for word in words[x.entity2_index:x.entity1_index]):
                    return 0
        # if mutant is close to either of the entities
        mutant_index = find_word_index(words, 'mutant')
        mutat_index = find_word_index(words, 'mutat')
        if (mutant_index != -1 and (abs(x.entity1_index - mutant_index) < 4 or abs(x.entity2_index - mutant_index) < 4)) or \
           (mutat_index != -1 and (abs(x.entity1_index - mutat_index) < 4 or abs(x.entity2_index - mutat_index) < 4)):
            return 0
    return ABSTAIN

#3
## Regulator
@labeling_function()
def lf_bind(x):
    return 1 if 'bind' in x.text.lower() else ABSTAIN
#4
@labeling_function()
def lf_interact(x):
    return 1 if 'interact' in x.text.lower() else ABSTAIN
#5
@labeling_function()
def lf_affinity(x):
    return 1 if 'affinit' in x.text.lower() else ABSTAIN
#6 TODO: 0.3578
## Upregulator
# Activator
@labeling_function()
def lf_activate(x):
    return 2 if 'activat' in x.text.lower() else ABSTAIN
#7
@labeling_function()
def lf_increase(x):
    return 2 if 'increas' in x.text.lower() else ABSTAIN
#8 TODO: 
@labeling_function()
def lf_induce(x):
    return 2 if 'induc' in x.text.lower() else ABSTAIN
#9 TODO: 
@labeling_function()
def lf_stimulate(x):
    return 2 if 'stimulat' in x.text.lower() else ABSTAIN
#10
@labeling_function()
def lf_upregulate(x):
    if ('upregulat' in x.text.lower() or 'up-regulat' in x.text.lower()) and ('downregulat' in x.text.lower() or 'down-regulat' in x.text.lower()):
        entity1_index = x.text.lower().index(x.entity1.lower())
        entity2_index = x.text.lower().index(x.entity2.lower())
        # if up regulate is between the two entities
        if isinstance(entity1_index, int) and isinstance(entity2_index, int):
            if entity1_index < entity2_index:
                if x.text[entity1_index:entity2_index].count('upregulat') > 0 or x.text[entity1_index:entity2_index].count('up-regulat') > 0:
                    return 2
            else:
                if x.text[entity2_index:entity1_index].count('upregulat') > 0 or x.text[entity2_index:entity1_index].count('up-regulat') > 0:
                    return 2
        return ABSTAIN
    else:
        return 2 if 'upregulat' in x.text.lower() or 'up-regulat' in x.text.lower() else ABSTAIN
#11
## Downregulator
@labeling_function()
def lf_downregulate(x):
    if('downregulat' in x.text.lower() or 'down-regulat' in x.text.lower()) and ('upregulat' in x.text.lower() or 'up-regulat' in x.text.lower()):
        if x.entity1_index == -1 or x.entity2_index == -1:
            return ABSTAIN
        # if up regulate is between the two entities
        if isinstance(x.entity1_index, int) and isinstance(x.entity2_index, int):
            if x.entity1_index < x.entity2_index:
                if x.text[x.entity1_index:x.entity2_index].count('downregulat') > 0 or x.text[x.entity1_index:x.entity2_index].count('down-regulat') > 0:
                    return 3
            else:
                if x.text[x.entity2_index:x.entity1_index].count('downregulat') > 0 or x.text[x.entity2_index:x.entity1_index].count('down-regulat') > 0:
                    return 3
        return ABSTAIN
    return 3 if 'downregulat' in x.text.lower() or 'down-regulat' in x.text.lower() else ABSTAIN
#12
@labeling_function()
def lf_reduce(x):
    return 3 if 'reduc' in x.text.lower() else ABSTAIN
#13
@labeling_function()
def lf_inhibit(x):
    return 3 if 'inhibit' in x.text.lower() else ABSTAIN
#14
@labeling_function()
def lf_decrease(x):
    return 3 if 'decreas' in x.text.lower() else ABSTAIN
#15
## Agonist
@labeling_function()
def lf_agonist(x):
    return 4 if ' agoni' in x.text.lower() or "\tagoni" in x.text.lower() else ABSTAIN

#16
## Antagonist
@labeling_function()
def lf_antagonist(x):
    return 5 if 'antagon' in x.text.lower() else ABSTAIN

#17
## Modulator
# TODO: Delete this LF, or change this to modulator ??
@labeling_function()
def lf_modulate(x):
    return 6 if 'modulat' in x.text.lower() else ABSTAIN

#18
@labeling_function()
def lf_allosteric(x):
    return 6 if 'allosteric' in x.text.lower() else ABSTAIN
#19
## Cofactor
@labeling_function()
def lf_cofactor(x):
    return 7 if 'cofactor' in x.text.lower() else ABSTAIN
#20
## Substrate/Product
@labeling_function()
def lf_substrate(x):
    return 8 if 'substrate' in x.text.lower() else ABSTAIN
#21
@labeling_function()
def lf_transport(x):
    return 8 if 'transport' in x.text.lower() else ABSTAIN
#22
@labeling_function()
def lf_catalyze(x):
    return 8 if 'catalyz' in x.text.lower() or 'catalys' in x.text.lower() else ABSTAIN
#23
@labeling_function()
def lf_product(x):
    return 8 if "produc" in x.text.lower() else ABSTAIN
#24
@labeling_function()
def lf_convert(x):
    return 8 if "conver" in x.text.lower() else ABSTAIN
#25
## NOT
@labeling_function()
def lf_not(x):
    entity1_index = x.text.lower().index(x.entity1.lower())
    entity2_index = x.text.lower().index(x.entity2.lower())
    # if the two entities are close to the word 'not'
    
    if 'not' in x.text.lower():
        if abs(entity1_index - x.text.lower().index('not')) < 20 or abs(entity2_index - x.text.lower().index('not')) < 20:
            return 9
        # if not is between the two entities
        if abs(entity1_index - x.text.lower().index('not')) < 40 or abs(entity2_index - x.text.lower().index('not')) < 40:
            if entity1_index < entity2_index:
                if x.text[entity1_index:entity2_index].count('not') > 0:
                    return 9
            else:
                if x.text[entity2_index:entity1_index].count('not') > 0:
                    return 9
    return ABSTAIN

# 26 replace the 17 (18)
@labeling_function()
def lf_combined_modulator(x):
    sentence_lower = x.text.lower()

    specific_terms = ['allosteric modulator', 'positive modulator', 'negative modulator', 'non-competitive modulator', 'positive allosteric modulator']
    if any(term in sentence_lower for term in specific_terms):
        return 6

    modulating_terms = ['modulat', 'allosteric', 'potentiate']
    for term in modulating_terms:
        if term in sentence_lower:
            term_index = sentence_lower.index(term)
            if x.entity1_index == -1 or x.entity2_index == -1:
                return ABSTAIN
            if abs(term_index - x.entity1_index) < 20 or abs(term_index - x.entity2_index) < 20:
                return 6
    # check the first indcidences of modulator and positive
    if x.entity1 in sentence_lower and x.entity2 in sentence_lower:
        entity1_index = sentence_lower.index(x.entity1.lower())
        entity2_index = sentence_lower.index(x.entity2.lower())
        between_entities = sentence_lower[min(entity1_index, entity2_index):max(entity1_index, entity2_index)]
        if 'modulate' in between_entities:
            return 6

    if 'positive' in sentence_lower and 'modulator' in sentence_lower:
        pos_indices = [i for i, word in enumerate(sentence_lower.split()) if word == 'positive']
        mod_indices = [i for i, word in enumerate(sentence_lower.split()) if 'modulator' in word]
        if len(pos_indices) == 0 or len(mod_indices) == 0:
            return ABSTAIN
        min_distance = min(abs(p - m) for p in pos_indices for m in mod_indices)
        if min_distance <= 3:
            return 6

    return ABSTAIN

lfs = [lf_amino_acid, lf_replace, lf_mutant, lf_bind, lf_interact, lf_affinity, lf_activate, lf_increase, lf_stimulate, lf_upregulate, lf_downregulate, lf_reduce, lf_inhibit, lf_decrease, lf_agonist, lf_antagonist, lf_combined_modulator, lf_allosteric, lf_cofactor, lf_substrate, lf_transport, lf_catalyze, lf_product, lf_convert, lf_not]

In [ ]:
dev_df = train_df.sample(250, random_state=123)

: 

In [ ]:
# show the first row of the dataframe
train_dev = train_df
train_dev = li.chemprot_enhanced(train_dev)
L_dev = li.apply_LFs(lfs, train_dev)

: 

In [30]:
# for i in range(train_dev.shape[0]):
#     if L_dev[i][2] == 0:
#         print(f"LFs: {L_dev[i]}")
#         print(f"Text: {train_dev.iloc[i].text}")
#         print(f"Entities: {train_dev.iloc[i].entity1}, {train_dev.iloc[i].entity2}")
#         print(f"True label: {train_dev.iloc[i].label}")
#         print("\n\n")

In [53]:
print("Test Coverage:", li.calc_coverage(L_dev))
lf_analysis = LFAnalysis(L_dev, lfs = lfs).lf_summary(Y = train_dev.label.values)

# Calculates how many of an LFs votes result in conflicts (helpful signal for debugging LFs)
lf_analysis['Conflict Ratio'] = lf_analysis['Conflicts'] / lf_analysis['Coverage']
lf_analysis

Test Coverage: 0.856232019283104


,j,Polarity,Coverage,Overlaps,Conflicts,Correct,Incorrect,Emp. Acc.,Conflict Ratio
lf_amino_acid,0,[0],0.020760,0.015318,0.014151,146,121,0.546816,0.681648
lf_replace,1,[0],0.002877,0.001555,0.001244,16,21,0.432432,0.432432
lf_mutant,2,[0],0.031102,0.023482,0.022160,84,316,0.210000,0.712500
lf_bind,3,[1],0.102092,0.077988,0.070057,760,553,0.578827,0.686215
lf_interact,4,[1],0.026670,0.019361,0.017106,181,162,0.527697,0.641399
lf_affinity,5,[1],0.045253,0.031335,0.025192,403,179,0.692440,0.556701
lf_activate,6,[2],0.115932,0.088096,0.070057,518,973,0.347418,0.604292
lf_increase,7,[2],0.098826,0.072623,0.062437,549,722,0.431943,0.631786
lf_induce,8,[2],0.154109,0.124796,0.106912,566,1416,0.285570,0.693744
lf_stimulate,9,[2],0.037633,0.032501,0.026437,137,347,0.283058,0.702479


In [55]:
# Calculate accuracy on the validation set (Ideally do this only at the end)
majority_model = MajorityLabelVoter(10)
preds_valid = majority_model.predict(L=L_dev)

print("acuracy for the not abstains")
print((preds_valid[preds_valid != -1] == train_dev[preds_valid != -1].label.values).mean())
print("acuracy for all")
print((preds_valid == train_dev.label.values).mean())

incorrect_indices = np.where(preds_valid!= train_dev.label.values)[0]
incorrect_predictions_df = train_dev.iloc[incorrect_indices]

acuracy for the not abstains
0.6460780049296796
acuracy for all
0.34647383562708967


In [56]:
# for i in range(incorrect_predictions_df.shape[0]):
#     print(L_dev[i])
#     print(incorrect_predictions_df.iloc[i].weak_labels)
#     print(incorrect_predictions_df.iloc[i].text)
#     print("True label:", incorrect_predictions_df.iloc[i].label)
#     print("Predicted label:", preds_valid[incorrect_indices[i]])
#     print("\n")

# Original label functions

In [57]:
# chemprot functions:

ABSTAIN = -1
### Keyword based labeling functions ###

## Part of
#0
@labeling_function()
def lf_amino_acid(x):
    return 0 if 'amino acid' in x.text.lower() else ABSTAIN
#1
@labeling_function()
def lf_replace(x):
    return 0 if 'replace' in x.text.lower() else ABSTAIN
#2
@labeling_function()
def lf_mutant(x):
    return 0 if 'mutant' in x.text.lower() or 'mutat' in x.text.lower() else ABSTAIN
#3
## Regulator
@labeling_function()
def lf_bind(x):
    return 1 if 'bind' in x.text.lower() else ABSTAIN
#4
@labeling_function()
def lf_interact(x):
    return 1 if 'interact' in x.text.lower() else ABSTAIN
#5
@labeling_function()
def lf_affinity(x):
    return 1 if 'affinit' in x.text.lower() else ABSTAIN
#6
## Upregulator
# Activator
@labeling_function()
def lf_activate(x):
    return 2 if 'activat' in x.text.lower() else ABSTAIN
#7
@labeling_function()
def lf_increase(x):
    return 2 if 'increas' in x.text.lower() else ABSTAIN
#8 
@labeling_function()
def lf_induce(x):
    return 2 if 'induc' in x.text.lower() else ABSTAIN
#9
@labeling_function()
def lf_stimulate(x):
    return 2 if 'stimulat' in x.text.lower() else ABSTAIN
#10
@labeling_function()
def lf_upregulate(x):
    return 2 if 'upregulat' in x.text.lower() else ABSTAIN
#11
## Downregulator
@labeling_function()
def lf_downregulate(x):
    return 3 if 'downregulat' in x.text.lower() or 'down-regulat' in x.text.lower() else ABSTAIN
#12
@labeling_function()
def lf_reduce(x):
    return 3 if 'reduc' in x.text.lower() else ABSTAIN
#13
@labeling_function()
def lf_inhibit(x):
    return 3 if 'inhibit' in x.text.lower() else ABSTAIN
#14
@labeling_function()
def lf_decrease(x):
    return 3 if 'decreas' in x.text.lower() else ABSTAIN
    
    
#15
## Agonist
@labeling_function()
def lf_agonist(x):
    return 4 if ' agoni' in x.text.lower() or "\tagoni" in x.text.lower() else ABSTAIN

#16
## Antagonist
@labeling_function()
def lf_antagonist(x):
    return 5 if 'antagon' in x.text.lower() else ABSTAIN

#17
## Modulator
# TODO: Delete this LF, or change this to modulator ??
@labeling_function()
def lf_modulate(x):
    return 6 if 'modulat' in x.text.lower() else ABSTAIN

#18
@labeling_function()
def lf_allosteric(x):
    return 6 if 'allosteric' in x.text.lower() else ABSTAIN
#19
## Cofactor
@labeling_function()
def lf_cofactor(x):
    return 7 if 'cofactor' in x.text.lower() else ABSTAIN
#20
## Substrate/Product
@labeling_function()
def lf_substrate(x):
    return 8 if 'substrate' in x.text.lower() else ABSTAIN
#21
@labeling_function()
def lf_transport(x):
    return 8 if 'transport' in x.text.lower() else ABSTAIN
#22
@labeling_function()
def lf_catalyze(x):
    return 8 if 'catalyz' in x.text.lower() or 'catalys' in x.text.lower() else ABSTAIN
#23
@labeling_function()
def lf_product(x):
    return 8 if "produc" in x.text.lower() else ABSTAIN
#24
@labeling_function()
def lf_convert(x):
    return 8 if "conver" in x.text.lower() else ABSTAIN
#25
## NOT
@labeling_function()
def lf_not(x):
    return 9 if 'not' in x.text.lower() else ABSTAIN

In [58]:
lfs = [lf_amino_acid, lf_replace, lf_mutant, lf_bind, lf_interact, lf_affinity, lf_activate, lf_increase, lf_induce, lf_stimulate, lf_upregulate, lf_downregulate, lf_reduce, lf_inhibit, lf_decrease, lf_agonist, lf_antagonist, lf_modulate, lf_allosteric, lf_cofactor, lf_substrate, lf_transport, lf_catalyze, lf_product, lf_convert, lf_not]

# show the first row of the dataframe
train_dev = dev_df
train_dev = li.chemprot_enhanced(train_dev)
L_dev2 = li.apply_LFs(lfs, train_dev)
L_dev2

100%|██████████| 250/250 [00:00<00:00, 8157.14it/s]


array([[-1, -1, -1, ..., -1, -1, -1],
       [-1, -1, -1, ..., -1, -1, -1],
       [-1, -1, -1, ..., -1, -1, -1],
       ...,
       [-1,  0, -1, ..., -1, -1, -1],
       [-1, -1, -1, ..., -1, -1,  9],
       [ 0, -1, -1, ..., -1, -1, -1]])

In [59]:
# difference = np.where(L_dev2 !=  L_dev)[0]
# count = 0
# for i in difference:
#     print(train_dev.iloc[i])
#     print(train_dev.iloc[i].text)
#     print(L_dev[i])
#     print(L_dev2[i])
#     count +=1
#     print("\n")
# print(count)

In [60]:
# Calculate accuracy on the validation set (Ideally do this only at the end)
majority_model = MajorityLabelVoter(10)
preds_valid = majority_model.predict(L=L_dev2)
print("Test Coverage:", li.calc_coverage(L_dev2))
lf_analysis = LFAnalysis(L_dev2, lfs = lfs).lf_summary(Y = train_dev.label.values)
# Calculates how many of an LFs votes result in conflicts (helpful signal for debugging LFs)
lf_analysis['Conflict Ratio'] = lf_analysis['Conflicts'] / lf_analysis['Coverage']
lf_analysis

Test Coverage: 0.888


,j,Polarity,Coverage,Overlaps,Conflicts,Correct,Incorrect,Emp. Acc.,Conflict Ratio
lf_amino_acid,0,[0],0.036,0.028,0.020,4,5,0.444444,0.555556
lf_replace,1,[0],0.004,0.000,0.000,0,1,0.000000,0.000000
lf_mutant,2,[0],0.016,0.016,0.008,1,3,0.250000,0.500000
lf_bind,3,[1],0.120,0.096,0.080,20,10,0.666667,0.666667
lf_interact,4,[1],0.028,0.024,0.016,5,2,0.714286,0.571429
lf_affinity,5,[1],0.036,0.024,0.016,5,4,0.555556,0.444444
lf_activate,6,[2],0.096,0.076,0.060,9,15,0.375000,0.625000
lf_increase,7,[2],0.068,0.052,0.044,5,12,0.294118,0.647059
lf_induce,8,[2],0.228,0.180,0.160,15,42,0.263158,0.701754
lf_stimulate,9,[2],0.048,0.028,0.024,1,11,0.083333,0.500000


In [62]:
print("acurracy: for the not abstain")
print((preds_valid[preds_valid != -1] == train_dev[preds_valid != -1].label.values).mean())
print("acurracy: for all the data")
print((preds_valid == train_dev.label.values).mean())
incorrect_indices = np.where(preds_valid[preds_valid != -1] != train_dev[preds_valid != -1].label.values)[0]
incorrect_predictions_df = train_dev.iloc[incorrect_indices]

acurracy: for the not abstain
0.6170212765957447
acurracy: for all the data
0.348


In [50]:
# for the whole trainning set

# DO NOT GO BEYOND THIS LINE

In [ ]:
new_train = li.chemprot_df_with_new_lf(train_df, lfs)
chemprot = li.df_to_chemprot(new_train)
li.save_dataset(chemprot, "../weak_datasets/chemprot2/train.json")
new_test = li.chemprot_df_with_new_lf(test_df, lfs)
chemprot = li.df_to_chemprot(new_test)
li.save_dataset(chemprot, "../weak_datasets/chemprot2/test.json")
new_valid = li.chemprot_df_with_new_lf(valid_df, lfs)
chemprot = li.df_to_chemprot(new_valid)
li.save_dataset(chemprot, "../weak_datasets/chemprot2/valid.json")

In [ ]:
# for particular functions
mutant, num = li.see_label_function(dev_df, [lf_activate])
print(num)
print(len(dev_df))
mutant = li.df_to_chemprot(mutant)
li.save_dataset(mutant, "./activate.json")

# Don't go beyond this line

## Main Work

In [14]:
df2.to_csv(f"./dev_{label}.csv")
chemprot6 = li.df_to_chemprot(li.chemprot_enhanced(label6))
li.save_dataset(chemprot6, f"./dev_{label}.json")


In [15]:
# Cell to inspect dev set for a given keyword (to inspect conflicts). Note: this doesn't work with "+" 
keyword = 'iPad' # Change this word
pd.set_option('display.max_colwidth', 1000)
dev_df[dev_df['text'].str.contains(keyword)]

,text,labels,entity1,entity2,span1,span2,weak_labels


## Evaluation

In [16]:
L_train = apply_LFs(lfs, train_df)
L_valid = apply_LFs(lfs, valid_df)
L_test = apply_LFs(lfs, test_df)

print("Train Coverage:", calc_coverage(L_train))
print("Valid Coverage:", calc_coverage(L_valid))
print("Test Coverage:", calc_coverage(L_test))

lf_analysis = LFAnalysis(L=L_valid, lfs=lfs).lf_summary()

# Calculates how many of an LFs votes result in conflicts (helpful signal for debugging LFs)
lf_analysis['Conflict Ratio'] = lf_analysis['Conflicts'] / lf_analysis['Coverage']
lf_analysis

NameError: name 'apply_LFs' is not defined

In [ ]:
# List LFs for which 'Conflict Ratio' is above some threshold (helpful for debugging)
lf_analysis[lf_analysis['Conflict Ratio'] > 0.8]['Conflict Ratio'].sort_values(ascending=False)

In [16]:
# Calculate accuracy on the validation set (Ideally do this only at the end)
majority_model = MajorityLabelVoter(10)
preds_valid = majority_model.predict(L)
(preds_valid[preds_valid != -1] == valid_df[preds_valid != -1].labels.values).mean()

NameError: name 'L_valid' is not defined

In [ ]:
json.dump(keywords, open("amazon_LFs_v1.json", "w"), indent=8)

In [ ]:
# Replace the LFs for a given dataset (in wrench format)
# dataset_name = "dbpedia"

# train_json = json.load(open(f"../weak_datasets/{dataset_name}/train.json", "r"))
# for idx in train_json:
#     train_json[idx]['weak_labels'] = [int(i) for i in list(L_train[int(idx)])]
    
# valid_json = json.load(open(f"../weak_datasets/{dataset_name}/valid.json", "r"))
# for idx in valid_json:
#     valid_json[idx]['weak_labels'] = [int(i) for i in list(L_valid[int(idx)])]
    
# test_json = json.load(open(f"../weak_datasets/{dataset_name}/test.json", "r"))
# for idx in test_json:
#     test_json[idx]['weak_labels'] = [int(i) for i in list(L_test[int(idx)])]

# json.dump(train_json, open(f"../weak_datasets/{dataset_name}/train.json", 'w'), indent=4)
# json.dump(valid_json, open(f"../weak_datasets/{dataset_name}/valid.json", 'w'), indent=4)
# json.dump(test_json, open(f"../weak_datasets/{dataset_name}/test.json", 'w'), indent=4)